# Extraction of dates from EHR

We'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [16]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.options.display.max_rows

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_colwidth', -1)

In [12]:
import pandas as pd
import numpy as np

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)


In [7]:
import pandas as pd
import numpy as np

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df2=pd.DataFrame(df, columns=['text'])

#Search for dates
#04/20/2009; 04/20/09; 4/20/09; 4/3/09
e1=df2['text'].str.extractall(r'(?P<month>(?:\d{1,2}))/(?P<day>(?:\d{1,2}))/(?P<year>(?:\d{4}|\d{2}))')
e1=e1.loc[pd.to_numeric(e1.day)<=31,:]
#Mar-20-2009; Mar 20, 2009; March 20, 2009; Mar. 20, 2009; Mar 20 2009;
e2=df2['text'].str.extractall(r'(?P<month2>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec))[a-z.\- ]*(?P<day2>(?:\d{1,2}))[,]* (?P<year2>(?:\d{4}))')
#Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
e3=df2['text'].str.extractall(r'(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec) (?:\d{1,2})(st|nd|th)+[, ]?(?:\d{2,4})')
#Feb 2009; Sep 2009; Oct 2010
e4=df2['text'].str.extractall(r'(?P<month4>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)) (?P<year4>(?:\d{2,4}))')
#6/2008; 12/2009
e5=df2['text'].str.extractall(r'(?P<month5>(?:[1][0-2]|\d{1}))/(?P<year5>(?:\d{4}))')
#2009; 2010
e6=df2['text'].str.extractall(r'(?P<year6>(?:1|2)\d{3})')
#20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
e7=df2['text'].str.extractall(r'(?P<e7>(?P<day7>(?:\d{1,2}) )?(?P<month7>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec))[a-z., ]*(?P<year7>(?:\d{4})))')

#merge above
df2.index.name='key'
e1.index.levels[0].name = 'key'
e2.index.levels[0].name = 'key'
e3.index.levels[0].name = 'key'
e4.index.levels[0].name = 'key'
e5.index.levels[0].name = 'key'
e6.index.levels[0].name = 'key'
e7.index.levels[0].name = 'key'
all=df2.merge(e1,how='left',on='key').merge(e2,
    how='left',on='key').merge(e3,
    how='left',on='key').merge(e4, 
    how='left',on='key').merge(e5, 
    how='left',on='key').merge(e6, 
    how='left',on='key').merge(e7, 
    how='left',on='key')
#impute letter months to numbers
month_dict={'Jan':1, 'Feb':2, 'Mar':3, 'Apr':4, 'May':5, 'Jun':6, 'Jul':7, 'Aug':8, 'Sep':9, 'Oct':10, 'Nov':11, 'Dec':12}
all.replace(dict(month=month_dict, month2=month_dict,  month4=month_dict, month7=month_dict), inplace=True)

#impute missing year, month, and day from e1
all.month.fillna(all.month7, inplace=True)
all.month.fillna(all.month2, inplace=True)
all.month.fillna(all.month4, inplace=True)
all.month.fillna(all.month5, inplace=True)
all.day.fillna(all.day7, inplace=True)
all.day.fillna(all.day2, inplace=True)
all.year.fillna(all.year7, inplace=True)
all.year.fillna(all.year2, inplace=True)
all.year.fillna(all.year4, inplace=True)
all.year.fillna(all.year5, inplace=True)
all.year.fillna(all.year6, inplace=True)
all1=all[['text','month', 'day', 'year']]
all2=all1.loc[:,('month', 'day', 'year')].apply(pd.to_numeric)

#impute missing month and day with 1
values = {'month': 1, 'day': 1}
all3=all2.fillna(value=values)

#convert 2 letter year to 1990's
all3.year=np.where(all3.year<1900,all3.year+1900,all3.year)

#sort by date
all3=all3.sort_values(by=['year','month','day'],na_position='last')
all4=all3.drop(['month','day','year'],axis=1).reset_index()
pd.Series(all4['key'], index=all4.index)

0        9
1       84
2        2
3       53
4       28
5      474
6      153
7       13
8      129
9       98
10     111
11     225
12      31
13     171
14     191
15     486
16     335
17     415
18      36
19     323
20     405
21     422
22     375
23     380
24     345
25      57
26     481
27     436
28     104
29     299
      ... 
470    320
471    383
472    244
473    286
474    480
475    431
476    279
477    198
478    381
479    463
480    366
481    255
482    439
483    401
484    475
485    257
486    152
487    235
488    464
489    253
490    231
491    427
492    141
493    186
494    161
495    413
496     25
497     39
498     40
499     71
Name: key, Length: 500, dtype: int64

In [22]:
    import numpy as np
    df2=pd.DataFrame(df, columns=['text'])

    #Search for dates
    #04/20/2009; 04/20/09; 4/20/09; 4/3/09
    e1=df2['text'].str.extractall(r'(?P<month>(?:\d{1,2}))/(?P<day>(?:\d{1,2}))/(?P<year>(?:\d{4}|\d{2}))')
    e1=e1.loc[pd.to_numeric(e1.day)<=31,:]
    #Mar-20-2009; Mar 20, 2009; March 20, 2009; Mar. 20, 2009; Mar 20 2009;
    e2=df2['text'].str.extractall(r'(?P<month2>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec))[a-z.,\- ]*(?P<day2>(?:\d{1,2}))[,]* (?P<year2>(?:\d{4}))')
    #Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
    #e3=df2['text'].str.extractall(r'(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec) (?:\d{1,2})(st|nd|th)+[, ]?(?:\d{2,4})')
    e3=df2['text'].str.extractall(r'(?P<month3>(?:\d{1,2}))-(?P<day3>(?:\d{1,2}))-(?P<year3>(?:\d{4}|\d{2}))')
    e3=e3.loc[pd.to_numeric(e3.day3)<=31,:]
    #Feb 2009; Sep 2009; Oct 2010
    e4=df2['text'].str.extractall(r'(?P<month4>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)) (?P<year4>(?:\d{2,4}))')
    #6/2008; 12/2009
    e5=df2['text'].str.extractall(r'(?P<month5>(?:[1][0-2]|\d{1}))/(?P<year5>(?:\d{4}))')
    #2009; 2010
    e6=df2['text'].str.extractall(r'(?P<year6>(?:1|2)\d{3})')
    #20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
    e7=df2['text'].str.extractall(r'(?P<e7>(?P<day7>(?:\d{1,2}) )?(?P<month7>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec))[a-z., ]*(?P<year7>(?:\d{4})))')

    #merge above
    e1.index=e1.index.droplevel(1)
    e2.index=e2.index.droplevel(1)
    e3.index=e3.index.droplevel(1)
    e4.index=e4.index.droplevel(1)
    e5.index=e5.index.droplevel(1)
    e6.index=e6.index.droplevel(1)
    e7.index=e7.index.droplevel(1)
    all=df2.merge(e1,how='left',left_index=True, right_index=True).merge(e2,
        how='left',left_index=True, right_index=True).merge(e3, 
        how='left',left_index=True, right_index=True).merge(e4, 
        how='left',left_index=True, right_index=True).merge(e5, 
        how='left',left_index=True, right_index=True).merge(e6, 
        how='left',left_index=True, right_index=True).merge(e7, 
        how='left',left_index=True, right_index=True) 
    #impute letter months to numbers
    month_dict={'Jan':1, 'Feb':2, 'Mar':3, 'Apr':4, 'May':5, 'Jun':6, 'Jul':7, 'Aug':8, 'Sep':9, 'Oct':10, 'Nov':11, 'Dec':12}
    all.replace(dict(month=month_dict, month2=month_dict,  month4=month_dict, month7=month_dict), inplace=True)

    #impute missing year, month, and day from e1
    all.month.fillna(all.month7, inplace=True)
    all.month.fillna(all.month2, inplace=True)
    all.month.fillna(all.month3, inplace=True)
    all.month.fillna(all.month4, inplace=True)
    all.month.fillna(all.month5, inplace=True)
    all.day.fillna(all.day7, inplace=True)
    all.day.fillna(all.day2, inplace=True)
    all.day.fillna(all.day3, inplace=True)
    all.year.fillna(all.year7, inplace=True)
    all.year.fillna(all.year3, inplace=True)
    all.year.fillna(all.year2, inplace=True)
    all.year.fillna(all.year4, inplace=True)
    all.year.fillna(all.year5, inplace=True)
    all.year.fillna(all.year6, inplace=True)
    all1=all[['text','month', 'day', 'year']]
    all2=all1.loc[:,('month', 'day', 'year')].apply(pd.to_numeric)

    #impute missing month and day with 1
    values = {'month': 1, 'day': 1}
    all3=all2.fillna(value=values)

    #convert 2 letter year to 1990's
    all3.year=np.where(all3.year<1900,all3.year+1900,all3.year)

    #sort by date
    all3=all3.sort_values(by=['year','month','day'],na_position='last')
    all4=all3.drop(['month','day','year'],axis=1).reset_index()
     
    all4
    #pd.Series(all4['index'], index=all4.index)

,index
0,9
1,84
2,2
3,53
4,28
5,474
6,153
7,13
8,129
9,98


In [17]:
all1

,text,month,day,year
0,03/25/93 Total time of visit (in minutes):\n,03,25,93
1,6/18/85 Primary Care Doctor:\n,6,18,85
2,sshe plans to move as of 7/8/71 In-Home Services: None\n,7,8,71
3,7 on 9/27/75 Audit C Score Current:\n,9,27,75
4,2/6/96 sleep studyPain Treatment Pain Level (Numeric Scale): 7\n,2,6,96
5,.Per 7/06/79 Movement D/O note:\n,7,06,79
6,"4, 5/18/78 Patient's thoughts about current substance abuse:\n",5,18,78
7,10/24/89 CPT Code: 90801 - Psychiatric Diagnosis Interview\n,10,24,89
8,3/7/86 SOS-10 Total Score:\n,3,7,86
9,(4/10/71)Score-1Audit C Score Current:\n,4,10,71


In [6]:
def date_sorter():
    import numpy as np
    df2=pd.DataFrame(df, columns=['text'])

    #Search for dates
    #04/20/2009; 04/20/09; 4/20/09; 4/3/09
    e1=df2['text'].str.extractall(r'(?P<month>(?:\d{1,2}))/(?P<day>(?:\d{1,2}))/(?P<year>(?:\d{4}|\d{2}))')
    e1=e1.loc[pd.to_numeric(e1.day)<=31,:]
    #Mar-20-2009; Mar 20, 2009; March 20, 2009; Mar. 20, 2009; Mar 20 2009;
    e2=df2['text'].str.extractall(r'(?P<month2>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec))[a-z.,\- ]*(?P<day2>(?:\d{1,2}))[,]* (?P<year2>(?:\d{4}))')
    #Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
    e3=df2['text'].str.extractall(r'(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec) (?:\d{1,2})(st|nd|th)+[, ]?(?:\d{2,4})')
    #Feb 2009; Sep 2009; Oct 2010
    e4=df2['text'].str.extractall(r'(?P<month4>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)) (?P<year4>(?:\d{2,4}))')
    #6/2008; 12/2009
    e5=df2['text'].str.extractall(r'(?P<month5>(?:[1][0-2]|\d{1}))/(?P<year5>(?:\d{4}))')
    #2009; 2010
    e6=df2['text'].str.extractall(r'(?P<year6>(?:1|2)\d{3})')
    #20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
    e7=df2['text'].str.extractall(r'(?P<e7>(?P<day7>(?:\d{1,2}) )?(?P<month7>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec))[a-z., ]*(?P<year7>(?:\d{4})))')

    #merge above
    df2.index.name= 'key'
    e1.index.name = 'key'
    e2.index.name = 'key'
    e3.index.name = 'key'
    e4.index.name = 'key'
    e5.index.name = 'key'
    e6.index.name = 'key'
    e7.index.name = 'key'
    all=df2.reset_index().merge(e1.reset_index().drop(['match'],axis=1),how='left',on='key').merge(e2.reset_index().drop(['match'],axis=1),
        how='left',on='key').merge(e4.reset_index().drop(['match'],axis=1), 
        how='left',on='key').merge(e5.reset_index().drop(['match'],axis=1), 
        how='left',on='key').merge(e6.reset_index().drop(['match'],axis=1), 
        how='left',on='key').merge(e7.reset_index().drop(['match'],axis=1), 
        how='left',on='key')
    #impute letter months to numbers
    month_dict={'Jan':1, 'Feb':2, 'Mar':3, 'Apr':4, 'May':5, 'Jun':6, 'Jul':7, 'Aug':8, 'Sep':9, 'Oct':10, 'Nov':11, 'Dec':12}
    all.replace(dict(month=month_dict, month2=month_dict,  month4=month_dict, month7=month_dict), inplace=True)

    #impute missing year, month, and day from e1
    all.month.fillna(all.month7, inplace=True)
    all.month.fillna(all.month2, inplace=True)
    all.month.fillna(all.month4, inplace=True)
    all.month.fillna(all.month5, inplace=True)
    all.day.fillna(all.day7, inplace=True)
    all.day.fillna(all.day2, inplace=True)
    all.year.fillna(all.year7, inplace=True)
    all.year.fillna(all.year2, inplace=True)
    all.year.fillna(all.year4, inplace=True)
    all.year.fillna(all.year5, inplace=True)
    all.year.fillna(all.year6, inplace=True)
    all1=all[['text','month', 'day', 'year']]
    all2=all1.loc[:,('month', 'day', 'year')].apply(pd.to_numeric)

    #impute missing month and day with 1
    values = {'month': 1, 'day': 1}
    all3=all2.fillna(value=values)

    #convert 2 letter year to 1990's
    all3.year=np.where(all3.year<1900,all3.year+1900,all3.year)

    #sort by date
    all3=all3.sort_values(by=['year','month','day'],na_position='last')
    all4=all3.drop(['month','day','year'],axis=1).reset_index()
    
    
    return pd.Series(all4['key'], index=all4.index)

In [7]:
date_sorter()

0        9
1       80
2        2
3       50
4       27
5       13
6       94
7      107
8       30
9       35
10      54
11     100
12      91
13      69
14     104
15      78
16      46
17      11
18       3
19      47
20      23
21      26
22      89
23      17
24     119
25      19
26     113
27      68
28     101
29       6
      ... 
470    470
471    471
472    472
473    473
474    474
475    475
476    476
477    477
478    478
479    479
480    480
481    481
482    482
483    483
484    484
485    485
486    486
487    487
488    488
489    489
490    490
491    491
492    492
493    493
494    494
495    495
496    496
497    497
498    498
499    499
Name: key, Length: 500, dtype: int64